ML4VA

Helen Yao, Eva Butler, and Larissa Cybyk

In [1]:
import pandas as pd
import numpy as np

C:\Users\l3n3c\AppData\Local\Temp\ipykernel_14936\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Data Cleaning & Visualization

loading/condensing crash data:

In [4]:
import datetime
try:
    crash_data = pd.read_csv('./crash_data.csv', sep='\t')
except:

    crash_data = pd.read_csv('./CrashData_test.csv')
    # crash_data.head()

    # crash_data.isnull().sum()

    crash_data = crash_data[['RTE Name','Crash Date']]

    # print(crash_data)

    crash_data['Crash Date'] = pd.to_datetime(crash_data['Crash Date'].str.split(' ', expand=True)[0], format='%m/%d/%Y')


    duplicate = crash_data.pivot_table(index = ['RTE Name', 'Crash Date'], aggfunc ='size')

    crash_data['Total Crashes'] = crash_data.set_index(['RTE Name', 'Crash Date']).index.map(duplicate)

    crash_data = crash_data.drop_duplicates()
    print(crash_data)

    # f = open("crash_data.csv", "a")
    crash_data.to_csv("crash_data.csv", sep='\t', encoding='utf-8')
    # f.close()

In [5]:
crash_data.head()

,Unnamed: 0,RTE Name,Crash Date,Total Crashes
0,0,R-VA IS00095NB,2016-12-06,15
1,1,S-VA114NP WOODLAND RD,2016-01-26,1
2,2,R-VA066SC00640NB,2016-02-13,1
3,3,R-VA042SC00782SB,2016-03-26,1
4,4,R-VA US00220NB,2016-07-18,1


loading/cleaning traffic volume data:

In [6]:
# load data

traffic_volume = pd.read_csv('./VDOT_Traffic_Volume.csv')

In [7]:
traffic_volume.head()

traffic_volume.columns
print(traffic_volume.shape)
nulls = traffic_volume.isnull().sum()
print(nulls)

(121679, 42)
OBJECTID                             0
LINKID                               0
DATA_DATE                            0
ROUTE_COMMON_NAME                    0
ROUTE_NAME                           0
HTRIS_ID                           745
ROUTE_ALIAS                          0
START_LABEL                          0
END_LABEL                            0
ADT                                  3
ADT_QUALITY                          0
PERCENT_4_TIRE                   94712
PERCENT_BUS                      94712
PERCENT_TRUCK_2_AXLE             94712
PERCENT_TRUCK_3_AXLE             94712
PERCENT_TRUCK_1_TRAIL            94712
PERCENT_TRUCK_2_TRAIL            94712
CLASS_QUALITY_CODE                   0
K_FACTOR                         92351
K_FACTOR_QUALITY_CODE               63
DIRECTION_FACTOR                 92371
AAWDT                            91950
AAWDT_QUALITY_CODE                   6
LOC_COMP_DIRECTIONALITY_NAME       196
ROUTE_FROM_MEASURE                   0
FROM_JURISDI

In [8]:
# dropping null columns / columns with a lot of missing data

to_drop = []
for col in traffic_volume.columns:
    if nulls[col] > 90000:
        to_drop.append(col)
print(to_drop)
traffic_volume = traffic_volume.drop(columns=to_drop)

['PERCENT_4_TIRE', 'PERCENT_BUS', 'PERCENT_TRUCK_2_AXLE', 'PERCENT_TRUCK_3_AXLE', 'PERCENT_TRUCK_1_TRAIL', 'PERCENT_TRUCK_2_TRAIL', 'K_FACTOR', 'DIRECTION_FACTOR', 'AAWDT', 'PEAK_SINGLE_UNIT_TRUCK', 'PEAK_COMBINATION_TRUCK', 'DATA_CUSTODIAN_EMAIL', 'METADATA_URL']


In [9]:
traffic_volume.isnull().sum()

OBJECTID                          0
LINKID                            0
DATA_DATE                         0
ROUTE_COMMON_NAME                 0
ROUTE_NAME                        0
HTRIS_ID                        745
ROUTE_ALIAS                       0
START_LABEL                       0
END_LABEL                         0
ADT                               3
ADT_QUALITY                       0
CLASS_QUALITY_CODE                0
K_FACTOR_QUALITY_CODE            63
AAWDT_QUALITY_CODE                6
LOC_COMP_DIRECTIONALITY_NAME    196
ROUTE_FROM_MEASURE                0
FROM_JURISDICTION                19
ROUTE_TO_MEASURE                  0
TO_JURISDICTION                  13
EVENT_SOURCE_ID                   0
EVENT_LOCATION_ID                 0
EVENT_COMPONENT_ID                0
MAXIMUMSCALEDENOMINATOR           9
ROUTE_COMMON_NAME_ADT             0
FROM_DISTRICT                   431
TO_DISTRICT                     544
RTE_TYPE_CD                       0
RTE_ID                      

In [19]:
# adding day of week feature
import re
import calendar

def convert_date(date, delim='/'):
    stripped = date.split(delim)
    try:
        given_date = datetime.datetime(int(stripped[2][:4]), int(stripped[0]), int(stripped[1]))
        
    except:
        given_date = datetime.datetime(int(stripped[0]), int(stripped[1]), int(stripped[2][:2]))
    return given_date
        
    
def day_of_week(given_date):
    day_num = given_date.weekday()
    day = 'sunday'
    if day_num == 0:
        day = 'monday'
    elif day_num == 1:
        day = 'tuesday'
    elif day_num == 2:
        day = 'wednesday'
    elif day_num == 3:
        day = 'thursday'
    elif day_num == 4:
        day = 'friday'
    elif day_num == 5:
        day = 'saturday'
    
    return day

def get_day(given_date):
    return given_date.day

def get_month(given_date):
    return given_date.month

def get_year(given_date):
    return given_date.year

** going to drop data_date once datasets are merged

In [11]:
traffic_volume['DATA_DATE'] = traffic_volume['DATA_DATE'].apply(convert_date) #converting data date into datetime

In [12]:
traffic_volume['DAY_WEEK'] = traffic_volume['DATA_DATE'].apply(day_of_week)
traffic_volume['DAY'] = traffic_volume['DATA_DATE'].apply(get_day)
traffic_volume['MONTH'] = traffic_volume['DATA_DATE'].apply(get_month)
traffic_volume['YEAR'] = traffic_volume['DATA_DATE'].apply(get_year)

In [13]:
traffic_volume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121679 entries, 0 to 121678
Data columns (total 33 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   OBJECTID                      121679 non-null  int64         
 1   LINKID                        121679 non-null  int64         
 2   DATA_DATE                     121679 non-null  datetime64[ns]
 3   ROUTE_COMMON_NAME             121679 non-null  object        
 4   ROUTE_NAME                    121679 non-null  object        
 5   HTRIS_ID                      120934 non-null  object        
 6   ROUTE_ALIAS                   121679 non-null  object        
 7   START_LABEL                   121679 non-null  object        
 8   END_LABEL                     121679 non-null  object        
 9   ADT                           121676 non-null  float64       
 10  ADT_QUALITY                   121679 non-null  object        
 11  CLASS_QUALITY

merge datasets BEFORE dropping

In [16]:
crash_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778666 entries, 0 to 778665
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     778666 non-null  int64 
 1   RTE Name       778666 non-null  object
 2   Crash Date     778666 non-null  object
 3   Total Crashes  778666 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 23.8+ MB


In [29]:
type(crash_data['Crash Date'].unique()[0])

pandas._libs.tslibs.timestamps.Timestamp

In [30]:
def convert_crash_date(date):
    if type(date) == pd._libs.tslibs.timestamps.Timestamp:
        return date
    stripped = date.split('-')
    try:
        given_date = datetime.datetime(int(stripped[2][:4]), int(stripped[0]), int(stripped[1]))
        
    except:
        given_date = datetime.datetime(int(stripped[0]), int(stripped[1]), int(stripped[2][:2]))
    return given_date

In [31]:
crash_data['Crash Date'] = crash_data['Crash Date'].apply(convert_crash_date)

In [35]:
crash_data.head()

,Unnamed: 0,RTE Name,Crash Date,Total Crashes
0,0,R-VA IS00095NB,2016-12-06,15
1,1,S-VA114NP WOODLAND RD,2016-01-26,1
2,2,R-VA066SC00640NB,2016-02-13,1
3,3,R-VA042SC00782SB,2016-03-26,1
4,4,R-VA US00220NB,2016-07-18,1


In [32]:
# merging
new_df = pd.merge(
    left=traffic_volume, 
    right=crash_data,
    how='left',
    left_on=['DATA_DATE', 'ROUTE_NAME'],
    right_on=['Crash Date', 'RTE Name']
)

In [ ]:
len(traffic_volume['DATA_DATE'].unique()) #2902 dates in traffic volume data

2902

In [ ]:
len(crash_data['Crash Date'].unique()) #3166 dates in traffic volume data

3166

In [46]:
len(set(traffic_volume['DATA_DATE']).intersection(crash_data['Crash Date'])) # only 1266 dates that overlap

1266

In [55]:
len(traffic_volume['ROUTE_NAME'].unique())

64941

In [56]:
len(crash_data['RTE Name'].unique())

59716

In [ ]:
len(set(traffic_volume['ROUTE_NAME']).intersection(crash_data['RTE Name'])) # 18766 routes overlap

18766

In [48]:
new_df.shape

(121679, 37)

In [49]:
new_df.isnull().sum()

OBJECTID                             0
LINKID                               0
DATA_DATE                            0
ROUTE_COMMON_NAME                    0
ROUTE_NAME                           0
HTRIS_ID                           745
ROUTE_ALIAS                          0
START_LABEL                          0
END_LABEL                            0
ADT                                  3
ADT_QUALITY                          0
CLASS_QUALITY_CODE                   0
K_FACTOR_QUALITY_CODE               63
AAWDT_QUALITY_CODE                   6
LOC_COMP_DIRECTIONALITY_NAME       196
ROUTE_FROM_MEASURE                   0
FROM_JURISDICTION                   19
ROUTE_TO_MEASURE                     0
TO_JURISDICTION                     13
EVENT_SOURCE_ID                      0
EVENT_LOCATION_ID                    0
EVENT_COMPONENT_ID                   0
MAXIMUMSCALEDENOMINATOR              9
ROUTE_COMMON_NAME_ADT                0
FROM_DISTRICT                      431
TO_DISTRICT              

dropping route identifiers, addresses, etc and double columns from merge

In [67]:
to_drop = ['ROUTE_COMMON_NAME', 'ROUTE_NAME', 'ROUTE_ALIAS', 'END_LABEL', 'START_LABEL', 'ADT_QUALITY', 'CLASS_QUALITY_CODE', 'K_FACTOR_QUALITY_CODE', 'AAWDT_QUALITY_CODE', 'MAXIMUMSCALEDENOMINATOR', 'ROUTE_COMMON_NAME_ADT','HTRIS_ID', 'Crash Date', 'RTE Name', 'Unnamed: 0', 'DATA_DATE', 'LINKID', 'OBJECTID']

In [68]:
combined_df = new_df.drop(columns=to_drop)

In [69]:
combined_df.head()

,ADT,LOC_COMP_DIRECTIONALITY_NAME,ROUTE_FROM_MEASURE,FROM_JURISDICTION,ROUTE_TO_MEASURE,TO_JURISDICTION,EVENT_SOURCE_ID,EVENT_LOCATION_ID,EVENT_COMPONENT_ID,FROM_DISTRICT,TO_DISTRICT,RTE_TYPE_CD,RTE_ID,Shape__Length,DAY_WEEK,DAY,MONTH,YEAR,Total Crashes
0,490.0,Bidirectional,0.000,Frederick County,0.24,Frederick County,795997,1,1,Staunton,Staunton,SC,2532756,478.381712,tuesday,7,6,2011,NaN
1,10.0,Bidirectional,0.620,Campbell County,0.64,Campbell County,709217,1,1,Lynchburg,Lynchburg,SC,1733670,47.494224,tuesday,10,12,2019,NaN
2,150.0,Bidirectional,3.180,Dickenson County,7.55,Dickenson County,720637,1,1,Bristol,Bristol,SC,1857885,8693.982351,monday,29,11,2021,NaN
3,47.0,Bidirectional,2.000,Nelson County,5.40,Nelson County,755648,1,1,Lynchburg,Lynchburg,SC,2111709,6516.476138,thursday,30,7,2020,NaN
4,30.0,Bidirectional,2.031,Washington County,2.08,Washington County,776797,1,1,Bristol,Bristol,SC,1594318,100.504177,friday,7,7,2017,NaN


In [70]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121679 entries, 0 to 121678
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   ADT                           121676 non-null  float64
 1   LOC_COMP_DIRECTIONALITY_NAME  121483 non-null  object 
 2   ROUTE_FROM_MEASURE            121679 non-null  float64
 3   FROM_JURISDICTION             121660 non-null  object 
 4   ROUTE_TO_MEASURE              121679 non-null  float64
 5   TO_JURISDICTION               121666 non-null  object 
 6   EVENT_SOURCE_ID               121679 non-null  int64  
 7   EVENT_LOCATION_ID             121679 non-null  int64  
 8   EVENT_COMPONENT_ID            121679 non-null  int64  
 9   FROM_DISTRICT                 121248 non-null  object 
 10  TO_DISTRICT                   121135 non-null  object 
 11  RTE_TYPE_CD                   121679 non-null  object 
 12  RTE_ID                        121679 non-nul

In [72]:
for feature in combined_df.select_dtypes("object"):
    print(feature, len(combined_df[feature].unique()))

LOC_COMP_DIRECTIONALITY_NAME 4
FROM_JURISDICTION 185
TO_JURISDICTION 185
FROM_DISTRICT 10
TO_DISTRICT 10
RTE_TYPE_CD 8
DAY_WEEK 7


Final combined dataframe:

In [73]:
combined_df.head()

,ADT,LOC_COMP_DIRECTIONALITY_NAME,ROUTE_FROM_MEASURE,FROM_JURISDICTION,ROUTE_TO_MEASURE,TO_JURISDICTION,EVENT_SOURCE_ID,EVENT_LOCATION_ID,EVENT_COMPONENT_ID,FROM_DISTRICT,TO_DISTRICT,RTE_TYPE_CD,RTE_ID,Shape__Length,DAY_WEEK,DAY,MONTH,YEAR,Total Crashes
0,490.0,Bidirectional,0.000,Frederick County,0.24,Frederick County,795997,1,1,Staunton,Staunton,SC,2532756,478.381712,tuesday,7,6,2011,NaN
1,10.0,Bidirectional,0.620,Campbell County,0.64,Campbell County,709217,1,1,Lynchburg,Lynchburg,SC,1733670,47.494224,tuesday,10,12,2019,NaN
2,150.0,Bidirectional,3.180,Dickenson County,7.55,Dickenson County,720637,1,1,Bristol,Bristol,SC,1857885,8693.982351,monday,29,11,2021,NaN
3,47.0,Bidirectional,2.000,Nelson County,5.40,Nelson County,755648,1,1,Lynchburg,Lynchburg,SC,2111709,6516.476138,thursday,30,7,2020,NaN
4,30.0,Bidirectional,2.031,Washington County,2.08,Washington County,776797,1,1,Bristol,Bristol,SC,1594318,100.504177,friday,7,7,2017,NaN


Transformation and model: